In [1]:
import numpy as np
from itertools import combinations

In [2]:

def modularity_matrix(adj_matrix: np.ndarray) -> np.ndarray:
    k_i = np.expand_dims(adj_matrix.sum(axis=1), axis=1)
    k_j = k_i.T
    norm = 1 / k_i.sum()
    K = norm * np.matmul(k_i, k_j)

    return norm * (adj_matrix - K)


In [3]:
def modularity(adj_matrix: np.ndarray, communities: list) -> float:

    k_i = np.expand_dims(adj_matrix.sum(axis=1), axis=1)
    k_j = k_i.T
    norm = 1 / k_i.sum() # 1/2m
    K = norm * np.matmul(k_i, k_j) # (ki * kj) / 2m
    mod_matrix = norm * (adj_matrix - K)  #  ( 1/2m ) *  Aij - (ki * kj) / 2m


    C = np.zeros_like(mod_matrix)

    for community in communities:
        for i, j in combinations(community, 2):
            C[i, j] = 1.0
            C[j, i] = 1.0

    return np.tril(np.multiply(mod_matrix, C), 0).sum()

In [4]:

def generate_adj_matrix(adj_matrix , V):
    # keep only the nodes in V
    for i in range(adj_matrix.shape[0]):
        if i not in V:
            adj_matrix[i, :] = 0
            adj_matrix[:, i] = 0
            
    return adj_matrix


In [5]:
def GCP(adj_matrix: np.ndarray) -> list:
    V = list(range(adj_matrix.shape[0]))
    
    v = np.random.choice(V)
    V.remove(v)
    
    K0 = [v]
    communities = [K0]

    while V:
        v = np.random.choice(V)
        Mdb = -1
        best_community = None
        best_community_index = -1

        # nodes  = communities.flatten()
        nodes = [i for community in communities for i in community]


        new_adj_matrix = generate_adj_matrix(
            adj_matrix, nodes + [v])


        for i,Ki in enumerate(communities):
            Ki_new =  Ki + [v] 
            
            new_communities = communities.copy()
      
            new_communities[i] = Ki_new

            Md = modularity( new_adj_matrix, new_communities )

            if Md > Mdb:
                Mdb = Md
                best_community = Ki_new
                best_community_index = i


        if len(nodes)>1:
            new_adj_matrix = generate_adj_matrix(adj_matrix, nodes)
            Mdphi = modularity(new_adj_matrix, communities)
        else: 
            Mdphi = -1 

        # TODO : verify if > or >=
        if Mdb > Mdphi :
            print("Md: ", Mdb)
            communities[best_community_index] = best_community
        else:
            print("Md: ", Mdphi)
            Ki = [v]           
            communities.append(Ki)  

        

        V.remove(v)

    return communities

In [6]:
adj_matrix = np.array([[0,1,1,1],
                        [1,0,1,1],
                        [1,1,0,1],
                        [1,1,1,0]
                        ])

generated_communities = GCP(adj_matrix)

Md:  0.25
Md:  0.25
Md:  0.25


In [7]:




# TODO : implementation of construction algorithm
def reconstruct(G):
    return G

# TODO : implementation of deconstruction algorithm
def destruct(G,beta):
    return G






def IG(G , nb_iterations = 100 , beta=.3):

    phi = GCP(G)

    for i in range(nb_iterations):
        phi_1 = destruct(phi , beta )
        
        phi_intermediate = reconstruct(phi_1 )

        if modularity(phi_intermediate) > modularity(phi):
            phi = phi_intermediate

    return phi